In [ ]:
import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy
print(numpy.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

print(torch.__version__)
print(torch.version.cuda)


In [ ]:
class PostcardDataset(Dataset):
    def __init__(self, csv_path, image_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.image_dir = image_dir
        self.transform = transform


        # Nur gültige Labels
        self.df = self.df.dropna(subset=["akon_id", "city", "country_id"])
        self.df = self.df[(self.df["city"].astype(str).str.strip() != "") & (self.df["country_id"].astype(str).str.strip() != "")]

        # Kombination aus Stadt und Land
        self.df["combo_label"] = self.df["city"] + " | " + self.df["country_id"]

        self.label_to_idx = {label: idx for idx, label in enumerate(self.df["combo_label"].unique())}
        self.idx_to_label = {idx: label for label, idx in self.label_to_idx.items()}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, row["akon_id"] + ".jpg")
        image = Image.open(image_path).convert("RGB")
        
        label = row["combo_label"]
        label_idx = self.label_to_idx[label]

        if self.transform:
            image = self.transform(image)

        return image, label_idx


In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


dataset = PostcardDataset("akon_postcards_public_domain.csv", "images/256", transform)
# dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)

# Beispielbild anzeigen
img, label_idx = dataset[0]
plt.imshow(img.permute(1, 2, 0) * 0.5 + 0.5)
plt.title(f"Label-Index: {label_idx}")
plt.show()


In [ ]:
embedding_dim = 32
z_dim = 100
num_labels = len(dataset.label_to_idx)

# Textembedding (z.B. Stadt)
label_embedding = nn.Embedding(num_labels, embedding_dim).to(device)


In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim, embedding_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(z_dim + embedding_dim, 1024 * 4 * 4),
            nn.ReLU(True)
        )
        
        self.conv = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, 4, 2, 1),  # 8x8
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1),  # 16x16
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1),  # 32x32
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1),  # 64x64
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 32, 4, 2, 1),   # 128x128
            nn.BatchNorm2d(32),
            nn.ReLU(True),

            nn.ConvTranspose2d(32, 3, 4, 2, 1),    # 256x256
            nn.Tanh()
        )

    def forward(self, z, label_embed):
        x = torch.cat([z, label_embed], dim=1)
        x = self.fc(x).view(-1, 1024, 4, 4)
        return self.conv(x)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.label_proj = nn.Linear(embedding_dim, 256 * 256)

        self.model = nn.Sequential(
            nn.Conv2d(4, 32, 4, 2, 1),  # 128x128
            nn.LeakyReLU(0.2),

            nn.Conv2d(32, 64, 4, 2, 1),  # 64x64
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, 4, 2, 1),  # 32x32
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 256, 4, 2, 1),  # 16x16
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 512, 4, 2, 1),  # 8x8
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(512, 1024, 4, 2, 1),  # 4x4
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2),

            nn.Flatten(),
            nn.Linear(1024 * 4 * 4, 1),
            nn.Sigmoid()
        )

    def forward(self, img, label_embed):
        label_map = self.label_proj(label_embed).view(-1, 1, 256, 256)
        x = torch.cat([img, label_map], dim=1)  # [B, 4, 256, 256]
        return self.model(x)


In [ ]:
generator = Generator(z_dim, embedding_dim).to(device)
discriminator = Discriminator(embedding_dim).to(device)

g_opt = torch.optim.Adam(generator.parameters(), lr=2e-4)
d_opt = torch.optim.Adam(discriminator.parameters(), lr=2e-4)

criterion = nn.BCELoss()


In [ ]:
import os
import time
import torch
from torchvision.utils import save_image

# Ordner vorbereiten
os.makedirs("training/DCGANv2/epoche_bilder", exist_ok=True)
os.makedirs("training/DCGANv2/epoche_schritte", exist_ok=True)

# Feste Inputs
fixed_noise = torch.randn(16, z_dim).to(device)
fixed_labels = torch.randint(0, num_labels, (16,)).to(device)

# Checkpoint laden, wenn vorhanden
start_epoch = 0
checkpoint_path = "training/DCGANv2/epoche_schritte/checkpoint_epoch_latest.pth"
if os.path.exists(checkpoint_path):
    print(" Checkpoint gefunden, lade...")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    generator.load_state_dict(checkpoint["generator_state_dict"])
    discriminator.load_state_dict(checkpoint["discriminator_state_dict"])
    g_opt.load_state_dict(checkpoint["g_opt_state_dict"])
    d_opt.load_state_dict(checkpoint["d_opt_state_dict"])
    start_epoch = checkpoint["epoch"]
    print(f" Fortsetzung ab Epoche {start_epoch}")
else:
    print(" Kein Checkpoint gefunden, starte bei Epoche 0.")

first = True
num_epochs = 10000  # Gesamtanzahl an Epochen

# Training-Schleife
for epoch in range(start_epoch, num_epochs):
    start_time = time.time()
    print(f"\n Starte Epoche {epoch+1}/{num_epochs}")
    batch_count = 0

    for imgs, label_idxs in dataloader:
        batch_count += 1

        imgs = imgs.to(device)
        label_idxs = label_idxs.to(device)
        batch_size = imgs.size(0)

        if first:
            z = torch.randn(batch_size, z_dim).to(device)
            print("Device imgs:", imgs.device)
            print("Device z:", z.device)
            print("Device generator:", next(generator.parameters()).device)
            print("Device discriminator:", next(discriminator.parameters()).device)
            first = False

        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # --- Train Discriminator ---
        z = torch.randn(batch_size, z_dim).to(device)
        label_embed = label_embedding(label_idxs)
        fake_imgs = generator(z, label_embed)

        real_preds = discriminator(imgs, label_embed)
        fake_preds = discriminator(fake_imgs.detach(), label_embed)
        d_loss = criterion(real_preds, real_labels) + criterion(fake_preds, fake_labels)

        d_opt.zero_grad()
        d_loss.backward()
        d_opt.step()

        # --- Train Generator ---
        z = torch.randn(batch_size, z_dim).to(device)
        label_embed = label_embedding(label_idxs)
        fake_imgs = generator(z, label_embed)
        fake_preds = discriminator(fake_imgs, label_embed)
        g_loss = criterion(fake_preds, real_labels)

        g_opt.zero_grad()
        g_loss.backward()
        g_opt.step()

    # Zeitmessung
    duration = time.time() - start_time
    print(f" Epoche {epoch+1} abgeschlossen | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f} | ⏱️ Zeit: {duration:.2f}s")

    # Alle 10 Epochen: Beispielbilder speichern
    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            embed = label_embedding(fixed_labels)
            samples = generator(fixed_noise, embed)
            save_image(samples * 0.5 + 0.5,
                       f"training/DCGANv2/epoche_bilder/gen_samples_epoch_{epoch+1}.png",
                       nrow=4)

    # Alle 100 Epochen: Checkpoint speichern
    if (epoch + 1) % 10 == 0:
        checkpoint_data = {
            'epoch': epoch + 1,
            'generator_state_dict': generator.state_dict(),
            'discriminator_state_dict': discriminator.state_dict(),
            'g_opt_state_dict': g_opt.state_dict(),
            'd_opt_state_dict': d_opt.state_dict(),
        }

        torch.save(checkpoint_data, f"training/DCGANv2/epoche_schritte/checkpoint_epoch_{epoch+1}.pth")
        torch.save(checkpoint_data, checkpoint_path)  # letzte Epoche überschreiben


In [ ]:
# Am Ende des Trainings (nach der Schleife)
torch.save(generator.state_dict(), "generator.pth")
torch.save(discriminator.state_dict(), "discriminator.pth")

z = torch.randn(1, z_dim)

In [ ]:
o = torch.ones_like(z)
o[:,0] = 5
z+=o
z = z.to(device)

In [ ]:
generator.eval()
with torch.no_grad():
    city_name = "Köln | DE"
    city_idx = torch.tensor([dataset.label_to_idx[city_name]]).to(device)
    label_embed = label_embedding(city_idx)
    
    gen_img = generator(z, label_embed).squeeze().detach().cpu()
    plt.imshow(gen_img.permute(1, 2, 0) * 0.5 + 0.5)
    plt.title(f"Generierte Postkarte: {city_name}")
    plt.axis("off")
    plt.show()


In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch
import pandas as pd

embedding_dim = 50
country_embedding = torch.nn.Embedding(len(all_countries), embedding_dim).to(device)

print(Counter(dataset.df["country_id"]).most_common(10))

all_countries = list(set(dataset.df["country_id"]))  #  Länder Redundanz nehmen
country_to_idx = {c: i for i, c in enumerate(all_countries)}  # Index für jedes Land

all_idxs = [country_to_idx[c] for c in all_countries]

embeddings = country_embedding(torch.tensor(all_idxs).to(device)).detach().cpu().numpy()

tsne = TSNE(n_components=2, random_state=42)
emb_2d = tsne.fit_transform(embeddings)

plt.figure(figsize=(12, 10))
plt.scatter(emb_2d[:, 0], emb_2d[:, 1])

# Beschriftung mit country_id
for i, country in enumerate(all_countries):
    plt.annotate(country, (emb_2d[i, 0], emb_2d[i, 1]), fontsize=8)

plt.title("Country Embeddings via t-SNE")
plt.show()


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
emb_2d = pca.fit_transform(embeddings)

plt.figure(figsize=(12, 10))
for country in df_tsne["country"].unique():
    subset = df_tsne[df_tsne["country"] == country]
    plt.scatter(subset["x"], subset["y"], label=country, s=10)

plt.title("Label Embeddings via PCA (farblich nach country_id)")
plt.legend(title="country_id", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize="small")
plt.tight_layout()
plt.show()
